In [2]:
import os, socket

socket.gethostname()

'DESKTOP-0IJAC10'

In [3]:
pwd

'c:\\Users\\dytpq0916\\VTO2025\\REFERENCES\\SewFormerAnalysis\\Sewformer\\ANALYSIS\\WINPC'

 'C:\Program Files\Autodesk\Maya2022\bin\mayapy.exe' -m IPython

 & 'C:\Program Files\Autodesk\Maya2022\bin\mayapy.exe'

In [5]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from glob import glob
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import json
import random
import pandas as pd
from tqdm import tqdm
import trimesh
import pickle
import pandas as pd
import torch

from ANALYSIS.analysis_utils import (
    plot_panel_info,
    visualize_meshes_plotly,
    filter_segmentation_map,
    filter_segmentation_map_clusters,
    is_clockwise,
)

from env_constants import SEWFORMER_PROJ_ROOT, DATASET_ROOT, PYGARMENT_ROOT

sys.path.append(PYGARMENT_ROOT)

import pygarment as pyg

In [6]:
garment_df = pd.read_csv("garment_df_20250215_165422.csv")

filtered_combination_name_list = np.unique(garment_df[
    garment_df["mesh_filter_failed"] == False
]["matching_combination_name"]).tolist()

filtered_combination_path_list = list(map(
    lambda x : os.path.join(DATASET_ROOT, 'sewfactory', x),
    filtered_combination_name_list
))

In [7]:
joints_name = (
    'Pelvis', 'L_Hip', 'R_Hip', 'Torso', 'L_Knee', 'R_Knee',
    'Spine', 'L_Ankle', 'R_Ankle', 'Chest', 'L_Toe', 'R_Toe',
    'Neck', 'L_Thorax', 'R_Thorax', 'Head', 'L_Shoulder', 'R_Shoulder',
    'L_Elbow', 'R_Elbow', 'L_Wrist', 'R_Wrist', 'L_Hand', 'R_Hand', 'Nose',
    'L_Eye', 'R_Eye', 'L_Ear', 'R_Ear', 'Head_top'
)
skeleton = (
    (0,1), (1,4), (4,7), (7,10), (0,2), (2,5), (5,8), (8,11), (0,3), (3,6), (6,9),
    (9,14), (14,17), (17,19), (19, 21), (21,23), (9,13), (13,16), (16,18), (18,20),
    (20,22), (9,12), (12,24), (24,15), (24,25), (24,26), (25,27), (26,28), (24,29)
)

In [ ]:
def amass_to_pose(src_path, save_path,save_path1):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    try:
        fps = bdata['mocap_framerate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    
    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    vertex_seq = []
    if bdata['gender'] == 'male':
        bm = male_bm
    else:
        bm = female_bm
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)
    root_data=0
    with torch.no_grad():
        for fId in range(0, frame_number, down_sample):
            root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
            #tt_test=root_orient[0][2].detach().clone()
            if fId==0:
                root_data=root_orient
            root_orient=root_orient-root_data
            #root_orient[0][1]=tt_test
            pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
            pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
            betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape

            #trans = torch.Tensor(bdata['trans'][fId:fId+1]).to(comp_device)    
            body = bm(pose_body=pose_body, pose_hand=pose_hand, betas=betas, root_orient=root_orient)
            joint_loc = body.Jtr[0] #+ trans
            vertex=body.v[0]
            pose_seq.append(joint_loc.unsqueeze(0))
            vertex_seq.append(vertex.unsqueeze(0))
    pose_seq = torch.cat(pose_seq, dim=0)
    vertex_seq = torch.cat(vertex_seq, dim=0).detach().cpu().numpy()
    
    pose_seq_np = pose_seq.detach().cpu().numpy()
    np.save(save_path, pose_seq_np)
    np.save(save_path1,vertex_seq)
    return fps